In [1]:
import numpy as np
import pandas as pd
from tkinter import filedialog as fd
from rdkit import Chem, DataStructs
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw, Descriptors, AllChem
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator

IPythonConsole.ipython_useSVG = True

In [2]:

import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score, matthews_corrcoef
import joblib

In [14]:

filename = fd.askopenfilename()
AHDL1Inhibitors = pd.read_csv(filename, header=None)

In [15]:
allTestedMolecules = AHDL1Inhibitors[0]  # firts 3 for testing, needs to change for all molecules (remove[0:4])
MolList = allTestedMolecules.values.tolist()
with open('AllTestedMols.txt', 'w') as fp:
    for mol in MolList:
        # write each item on a new line
        fp.write("%s\n" % mol)
    print('Done')

Done



### Generate Fingerprints

In [16]:

#x = all data from Principal components 1:61
x = AHDL1Inhibitors.iloc[:, 2:-2]
x.shape

(1001, 60)

In [17]:

#y is inhibition score
# check imbalance dataset:
y = AHDL1Inhibitors.iloc[:, -1]
#sum(y)/len(y)
print(y)
# mild imbalance

0       ALDH1_inhibition
1                      1
2                      1
3                      1
4                      1
              ...       
996                    0
997                    0
998                    0
999                    0
1000                   0
Name: 63, Length: 1001, dtype: object


### machine learning:
Random forrest

In [18]:

# split data in train and test sets. Set the testset size to 20%
seed = 13
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.20, random_state=seed)
# create folds for cross- validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [19]:
# print out ids of folds
for i, (train_index, test_index) in enumerate(cv.split(xTrain, yTrain)):
    print("\nFold_" + str(i + 1))
    print("TRAIN:", train_index)
    print("TEST:", test_index)


Fold_1
TRAIN: [  0   1   2   3   4   5   7   8   9  11  12  13  14  15  16  17  19  20
  21  22  23  25  26  28  29  30  31  32  33  34  35  37  38  39  40  41
  42  43  44  45  47  49  50  51  52  53  54  55  57  58  59  60  61  62
  63  64  65  67  69  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  93  94  95  96  98 100 101 102 103 104 105
 106 107 108 109 110 112 114 115 117 118 119 121 122 123 124 125 128 129
 133 134 136 137 138 139 140 141 142 143 144 145 146 147 148 149 152 153
 154 155 156 157 158 159 160 161 162 164 165 166 167 168 170 171 172 173
 175 176 177 178 180 181 182 183 184 185 186 188 189 190 191 192 193 195
 196 198 199 200 201 202 205 206 207 208 209 210 211 212 214 215 216 217
 218 219 221 223 224 225 227 228 229 230 231 234 235 237 238 240 241 242
 243 244 245 246 250 251 252 253 254 255 257 258 259 260 262 263 264 268
 270 271 272 273 277 278 280 281 282 284 286 287 288 289 290 293 295 297
 298 299 301 303 304 305 306 307 308

C:\Users\Gebruiker\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [20]:
# Scale inputs
scale = StandardScaler().fit(xTrain)
xTrain = scale.transform(xTrain)


In [21]:
# save data for future use
joblib.dump(scale, "Fingerprints.pkl", compress=3)

['Fingerprints.pkl']

In [22]:
# create grid search dictionary
param_grid = {"max_features": [xTrain.shape[1] // 10, xTrain.shape[1] // 7,
                               xTrain.shape[1] // 5, xTrain.shape[1] // 3],
              "n_estimators": [100, 250, 500]}

In [23]:
# setup model building
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [24]:
# run model building
m.fit(xTrain, yTrain)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\Gebruiker\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=13, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=2,
             param_grid={'max_features': [6, 8, 12, 20],
                         'n_estimators': [100, 250, 500]},
             verbose=1)

In [25]:
m.best_params_
m.best_score_
m.cv_results_


{'mean_fit_time': array([0.20936689, 0.52208009, 1.03649211, 0.24770832, 0.61633277,
        1.23177481, 0.32530451, 0.81632376, 1.63340402, 0.4888484 ,
        1.21249347, 2.48173699]),
 'std_fit_time': array([0.00172611, 0.00564903, 0.00698236, 0.00268076, 0.00672135,
        0.01218741, 0.00587268, 0.00574966, 0.01170407, 0.0109757 ,
        0.01404691, 0.04207784]),
 'mean_score_time': array([0.01753645, 0.01895361, 0.03641477, 0.00777349, 0.01894975,
        0.03735285, 0.00778518, 0.01876273, 0.03892107, 0.00798459,
        0.01896911, 0.03906045]),
 'std_score_time': array([1.09092201e-02, 6.39418119e-04, 8.87463150e-04, 3.96332595e-04,
        8.98898727e-04, 1.53047212e-03, 7.48066605e-04, 1.15497875e-03,
        1.66951433e-03, 1.13517292e-05, 8.69897955e-04, 3.35220443e-03]),
 'param_max_features': masked_array(data=[6, 6, 6, 8, 8, 8, 12, 12, 12, 20, 20, 20],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, 

In [26]:
joblib.dump(m, "RFmodelMorganFingerprint.pkl", compress=3)

['RFmodelMorganFingerprint.pkl']

Load model(also to check if it works)

In [27]:
scale = joblib.load("Fingerprints.pkl")
# scale descriptors of the test set compounds
xTest = scale.transform(xTest)
# predict logBB class
predRF = m.predict(xTest)
predRF

array(['0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0',
       '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0

In [28]:
accuracy_score(yTest, predRF)

0.7164179104477612

In [29]:
# if the model includes several ones like RF models or consensus models (or for probabilistic models)
# we can calculate consistency of predictions amongs those models and use it for estimation of applicability domain
pred_prob = m.predict_proba(xTest)
pred_prob

array([[0.628, 0.372, 0.   ],
       [0.656, 0.344, 0.   ],
       [0.728, 0.272, 0.   ],
       [0.746, 0.254, 0.   ],
       [0.668, 0.332, 0.   ],
       [0.49 , 0.51 , 0.   ],
       [0.668, 0.332, 0.   ],
       [0.656, 0.344, 0.   ],
       [0.494, 0.504, 0.002],
       [0.658, 0.342, 0.   ],
       [0.49 , 0.51 , 0.   ],
       [0.928, 0.072, 0.   ],
       [0.754, 0.246, 0.   ],
       [0.662, 0.338, 0.   ],
       [0.852, 0.148, 0.   ],
       [0.72 , 0.28 , 0.   ],
       [0.65 , 0.35 , 0.   ],
       [0.642, 0.358, 0.   ],
       [0.86 , 0.14 , 0.   ],
       [0.664, 0.336, 0.   ],
       [0.632, 0.368, 0.   ],
       [0.648, 0.352, 0.   ],
       [0.808, 0.192, 0.   ],
       [0.676, 0.322, 0.002],
       [0.372, 0.628, 0.   ],
       [0.79 , 0.21 , 0.   ],
       [0.722, 0.278, 0.   ],
       [0.688, 0.312, 0.   ],
       [0.618, 0.382, 0.   ],
       [0.796, 0.204, 0.   ],
       [0.79 , 0.21 , 0.   ],
       [0.696, 0.304, 0.   ],
       [0.672, 0.328, 0.   ],
       [0.

In [30]:
# setup threshold
threshold = 0.8

In [31]:
# calc maximum predicted probability for each row (compound) and compare to the threshold
da = np.amax(pred_prob, axis=1) > threshold
da

array([False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False,  True, False, False, False,
        True, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False,  True,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False,  True,  True,  True, False,  True, False, False, False,
        True, False, False, False, False, False, False, False,  True,
       False, False,  True, False, False,  True, False, False, False,
       False,  True, False, False, False, False,  True, False, False,
       False,  True, False, False, False, False, False,  True, False,
       False, False,

In [32]:
# calc statistics
accuracy_score(np.asarray(yTest)[da], predRF[da])

0.9393939393939394

In [33]:
# calc coverage
sum(da) / len(da)

0.16417910447761194

### Machine learning:
svm

In [ ]:
# create grid search dictionary
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}
# setup model building
svm = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid, n_jobs=2, cv=cv, verbose=1)

In [ ]:
# run model building
svm.fit(xTrain, yTrain)

In [ ]:
svm.best_params_

In [ ]:
svm.best_score_

In [ ]:
# save model
joblib.dump(svm, "SVMmorganFingerprint.pkl", compress=3)

In [ ]:
# predict logBB for the test set compounds
pred_svm = svm.predict(xTest)
pred_svm


In [ ]:
# calc statistics
print("Accuracy = ", accuracy_score(yTest, pred_svm))
print("MCC = ", matthews_corrcoef(yTest, pred_svm))
print("Kappa = ", cohen_kappa_score(yTest, pred_svm))

In [ ]:
# estimate applicability domain and calc stat
pred_prob = svm.predict_proba(xTest[:, :60])
pred_prob

In [ ]:
da = np.amax(pred_prob, axis=1) > threshold

In [ ]:
print("Accuracy = ", accuracy_score(np.asarray(yTest)[da], pred_svm[da]))
print("MCC = ", matthews_corrcoef(np.asarray(yTest)[da], pred_svm[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(yTest)[da], pred_svm[da]))
print("Coverage = ", sum(da) / len(da))


In [ ]:
#xNew = np.concatenate((x,allDescrsDf), axis=1)

xNew = x

In [ ]:
xNtr, xNts, yNtr, yNts = train_test_split(xNew, y, test_size=0.20, random_state=seed)
scale = StandardScaler().fit(xNtr)
xNtr = scale.transform(xNtr)

In [ ]:
# create grid search dictionary
param_grid = {"max_features": [xNtr.shape[1] // 10, xNtr.shape[1] // 7, xNtr.shape[1] // 5, xNtr.shape[1] // 3],
              "n_estimators": [100, 250, 500]}

In [ ]:
# setup model building
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [ ]:
# run model building
m.fit(xNtr, yNtr)

In [ ]:
m.best_score_

In [ ]:
# scale descriptors of the test set compounds
xNts = scale.transform(xNts)
# predict
pred = m.predict(xNts)
pred

In [ ]:
# calc statistics
print("Accuracy = ", accuracy_score(yNts, pred))
print("MCC = ", matthews_corrcoef(yNts, pred))
print("Kappa = ", cohen_kappa_score(yNts, pred))

In [ ]:
# estimate applicability domain and calc stat
pred_prob = m.predict_proba(xNts)
da = np.amax(pred_prob, axis=1) > threshold

print("Accuracy = ", accuracy_score(np.asarray(yNts)[da], pred[da]))
print("MCC = ", matthews_corrcoef(np.asarray(yNts)[da], pred[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(yNts)[da], pred[da]))
print("Coverage = ", sum(da) / len(da))

In [ ]:
# rebuild RF model manually using best parameters to be able to extract additional information from the model
rf = RandomForestClassifier(n_estimators=m.best_params_["n_estimators"],
                            max_features=m.best_params_["max_features"],
                            random_state=seed)
rf.fit(xNtr, yNtr)

In [ ]:
imp = rf.feature_importances_
imp

In [ ]:
indices = np.argsort(imp)[::-1]

print("Feature ranking:")

# print top 10 features
for i in range(10):
    print("%d. feature %d (%f)" % (i + 1, indices[i], imp[indices[i]]))

### Apply model to new data (Random forrest)

In [40]:
filename = fd.askopenfilename()
AHDL1Inhibitors2 = pd.read_csv(filename, header=None)

In [41]:
allTestedMolecules = AHDL1Inhibitors2[0]  # firts 3 for testing, needs to change for all molecules (remove[0:4])
MolList = allTestedMolecules.values.tolist()
with open('AllTestedMols.txt', 'w') as fp:
    for mol in MolList:
        # write each item on a new line
        fp.write("%s\n" % mol)
    print('Done')

Done


In [42]:
x2 = AHDL1Inhibitors.iloc[:, 2:-2]
x2.shape

(1001, 60)

In [43]:
#y is inhibition score
# check imbalance dataset:
y2 = AHDL1Inhibitors.iloc[:, -1]
#sum(y)/len(y)
print(y2)
# mild imbalance

0       ALDH1_inhibition
1                      0
2                      0
3                      0
4                      0
              ...       
996                    1
997                    1
998                    1
999                    1
1000                   1
Name: 63, Length: 1001, dtype: object


In [44]:
Prediction = m.predict(x2)
Prediction

array(['ALDH1_inhibition', '0', '1', ..., '0', '1', '1'], dtype=object)

### Apply model to new data (SVM)

In [ ]:
# predict logBB for the test set compounds
pred_svm2 = svm.predict(x2)
pred_svm2

In [ ]:
# calc statistics
print("Accuracy = ", accuracy_score(y2, pred_svm2))
print("MCC = ", matthews_corrcoef(y2, pred_svm2))
print("Kappa = ", cohen_kappa_score(y2, pred_svm2))

In [ ]:
# estimate applicability domain and calc stat
pred_prob = svm.predict_proba(x2[:,:60])
pred_prob

In [ ]:
da = np.amax(pred_prob, axis=1) > threshold

In [ ]:
print("Accuracy = ", accuracy_score(np.asarray(y2)[da], pred_svm[da]))
print("MCC = ", matthews_corrcoef(np.asarray(y2)[da], pred_svm[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(y2)[da], pred_svm[da]))
print("Coverage = ", sum(da) / len(da))